In [ ]:
# Load the required packages
import os
from PIL import Image
from tqdm import tqdm
import shutil

In [ ]:
# Here we define the main function of this script, "split_image_and_labels" which is intended to split the images in three first vertically and then horisontally 
# Beyond splitting the images, it also loops through all labels which are associated with the current image and then recalculates them to fit the crop they now belong to
# Since the bounding boxes are defined relative to the height and width of the image. This function takes the image, labels, direction (of the crop), output_prefix, and start_index.
# The final two are just used for the purpose of naming the tiled images when saving.
def split_image_and_labels(img, labels, direction, output_prefix, start_index=0):
    # Get the width and height of the current image
    width, height = img.size
    # Make an empty list
    pieces = []
    # Create a for loop which runs 3 times, and where i is equal to 0, 1, and 2 in turn.
    for i in range(3):
        # If the direction is specified as vertical (for the vertical crops), this part runs.
        if direction == "vertical":
            # Since images in YOLO start with the coordinate system in the top left corner, y increases downward and x increases to the right. The top variable for i = 0 is then
            # 0 * (1/3 * height of the image) which is just 0 on the y-axis. Then as i increases we get 1 * (1/3 * height) which is then 1/3 thirds of the way etc.
            top = i * (height // 3)
            # Then we also have the bottom variable, which determines how far down the split should go in the i'th iteration of the loop. Saying i = 0 again, we get 1 * (1/3 * height)
            # Meaning the top is at y = 0, and the bottom is 1/3 of the height. Corresponding to roughly a third of the image
            # This goes as long as i < 2, after which the top will be 2/3 or the height and the bottom will be the full height. 
            bottom = (i + 1) * (height // 3) if i < 2 else height
            # This follows the format: left, top, right, bottom), which means it starts at left = 0 (x = 0) and covers the until right = width. So the full width of the image.
            # Then it takes the top and bottom from before, and now we have all the values needed to cut out a section of the image.
            box = (0, top, width, bottom)
            # If the direction isn't vertical, we do the same thing but horizontally. 
        else:
            left = i * (width // 3)
            right = (i + 1) * (width // 3) if i < 2 else width
            box = (left, 0, right, height)
        # We use the img.crop function to create a tile matching the box variable we just created.
        tile = img.crop(box)
        # Here we initially an empty list which will hold the new labels for the current (newly created) tile
        new_tile_labels = []
        # Loop through the list of labels which will be defined later
        for label in labels:
            # From each label split the following values into five seperate values. Class, x, y, width, and height. These values are further explained in the thesis, section 3
            # They are however all relative to the height and width of the image (except "cls" which is just a number representing which class the label belongs to)
            cls, x, y, w, h = label.strip().split()
            # Convert all the four values from strings to float values, so that we can manipulate them
            x, y, w, h = map(float, [x, y, w, h])
            # Here we do the same thing to all the values, which is converting them to absolute pixel values. We do this, so that we can check whether a label belongs to the
            # new crop we have made. 
            abs_x = x * width
            abs_y = y * height
            abs_w = w * width
            abs_h = h * height
            # If the direction is vertical,
            if direction == "vertical":
                # and the y value (center of the bounding box) is not greater than or equal to the top
                # while being less than or equal to the bottom (Center of the bounding box is not above or below the crop borders)
                if not (top <= abs_y <= bottom):
                    # then continue
                    continue
                # Since we still want the values to be in the YOLO format, we now have to convert all labels which
                # fall inside our new crop back to the relative values they originally had 
                new_y = abs_y - top
                new_x = abs_x
                # " /= " is division assignment, so we are basically dividing the variable by (bottom - top) and then assigning the result to the variable
                new_y /= (bottom - top)
                new_x /= width
                new_h = abs_h / (bottom - top)
                new_w = abs_w / width
            # If the direction is not vertical we essentially do the same thing, but horisontially. The same logic as before applies
            else:
                # Obviously we now have to instead check whether the x-value of the center of the bounding box falls within our left and right borders in the crop
                if not (left <= abs_x <= right):
                    continue
                # If it was within the border, we can calculate the relative positions
                new_x = abs_x - left
                new_y = abs_y
                new_x /= (right - left)
                new_y /= height
                new_w = abs_w / (right - left)
                new_h = abs_h / height
            # Append the new values in the same as before with class, x, y, w, h. Values are rounded to 17 decimals, which is the same as the original dataset
            new_tile_labels.append(f"{cls} {new_x:.17f} {new_y:.17f} {new_w:.17f} {new_h:.17f}")
        # Append the tile, list of labels, and name of the new tile to the pieces list we created earlier
        pieces.append((tile, new_tile_labels, f"{output_prefix}_{start_index + i}"))
    return pieces

# Here we create the process_split function, which is designed to help us loop through each split (train, test, val + corresponding label folders) in the original folder 
# by using the "split" argument and then create folders with matching names in our new dataset folder "tiled_dataset", which we can save our cropped images to
def process_split(split):
    # Input directories
    img_dir = os.path.join("training_data", "images", split)
    lbl_dir = os.path.join("training_data", "labels", split)
    # Output directories, images and labels
    out_img_dir = os.path.join("tiled_dataset", "images", split)
    out_lbl_dir = os.path.join("tiled_dataset", "labels", split)
    # If the directories doesn't exist, create them with the name "tiled_dataset/images/split" and "tiled_dataset/labels/split"
    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_lbl_dir, exist_ok=True)

    # Create a list of image files by looping through the files in the image directory 
    image_files = [f for f in os.listdir(img_dir)]

    # Loop through all files in the image files list, while using tqdm to create a loading bar which also states which split we are currently cropping
    for img_file in tqdm(image_files, desc=f"Processing {split}"):
        # Get the base name of each image by removing the ".jpg"
        base = os.path.splitext(img_file)[0]
        # Create the path to the current image 
        img_path = os.path.join(img_dir, img_file)
        # Create the path to the corresponding label by going into the label directory and opening the image name + .txt
        label_path = os.path.join(lbl_dir, f"{base}.txt")

        # Make sure the script don't crash if there is an image without a matching label file
        if not os.path.exists(label_path):
            continue
        
        # Open the image in the image path 
        img = Image.open(img_path)
        # Open current the label file in the label_path 
        with open(label_path, 'r') as f:
            # Use .readlines() to create a list of all labels in the corresponding image
            labels = f.readlines()

        # The v_tiles variable will hold the vertical tiles/crops when running the split_image_and_labels function with the "vertical" argument
        # These will be a tuple consisting of the image, labels and name
        v_tiles = split_image_and_labels(img, labels, "vertical", f"{base}_v")
        # Unpack every image, label and name in the previous tuple
        for v_img, v_lbls, v_name in v_tiles:
            # Use the split_image_and_labels function to horizontally split all of the vertical crops 
            h_tiles = split_image_and_labels(v_img, v_lbls, "horizontal", v_name, 0)
            # Unpack the previous tuple 
            for final_img, final_lbls, final_name in h_tiles:
                # Save the finished crop in the output directory and add ".jpg" back to the name
                final_img.save(os.path.join(out_img_dir, f"{final_name}.jpg"))
                # Open the output label directory and save the label corresponding to the image in the previous step while adding ".txt" to the name
                with open(os.path.join(out_lbl_dir, f"{final_name}.txt"), 'w') as f:
                    f.write('\n'.join(final_lbls))

# Since we still need a dataset.yaml file to run our model we create a create_dataset_yaml function capable of generating one based on the inputs "output_path" and "class_names"
def create_dataset_yaml(output_path, class_names):
    with open(output_path, 'w') as f:
        # Write the paths to the individual folders needed for training into the dataset.yaml file
        f.write(f"path: tiled_dataset\n")
        f.write("train: images/train\n")
        f.write("val: images/val\n")
        f.write("test: images/test\n")
        # Write the names of the classes into the dataset.yaml file by looping through the names provided in a list and mapping them like 0: "Carrot", 1: "Cross"
        f.write("names:\n")
        for i, name in enumerate(class_names):
            f.write(f"  {i}: {name}\n")

# Def a function in which we can specify the final values 
def main():
    # Class names in our dataset
    class_names = ["Carrot", "Cross"]
    # Loop through the desired splits we want to crop and run process_split on each
    for split in ["train", "val", "test"]:
        process_split(split)
    # Run the create_dataset_yaml file and specify the path for the dataset.yaml file and the class names
    create_dataset_yaml("tiled_dataset/dataset.yaml", class_names)

# Run the function "main()"
if __name__ == "__main__":
    main()